# Nội dung

1. Tổng quan
2. Attention Mechanism
3. Thuật toán
4. Ví dụ


# Các tài liệu tham khảo:
1. NEURAL MACHINE TRANSLATION BY JOINTLY LEARNING TO ALIGN AND TRANSLATE: https://arxiv.org/pdf/1409.0473.pdf
2. http://itechseeker.com/tutorials/nlp-with-deep-learning/ly-thuyet-chung/attention-mechanism/
3. http://itechseeker.com/tutorials/nlp-with-deep-learning/ly-thuyet-chung/sequence-to-sequence-model/
4. Nature Language Processing in action
5. Sequence to Sequence Learning with Neural Networks - Ilya Sutskever, Oriol Vinyals, Quoc V. Le

# 1. Tổng quan

Việc huấn luyện cho máy là việc rất khó khăn. Deep Neural Networks là những mô hình mạnh đã đạt được hiệu suất rất tuyệt vời trong việc giúp cho máy học hiểu. Mặc dù mô hình DNNs hoạt động tốt bất cứ khi nào có số lượng lớn dữ liệu training được gán nhãn. Các mô hình DNNs được áp dụng với các bài toán có input là các vecto được mã hóa có đầu vào cố định, nhưng các mô hình này lại không thể ánh xạ từ chuỗi sang chuỗi. Đó là hạn chế đáng kể của mô hình DNNs, vì có nhiều vấn đề quan trọng được thể hiện ở các chuỗi có độ dài không xác định.



Mô hình Sequence to Sequence (Seq2Seq) đã được nghiên cứu ra để cải thiện hạn chế nói trên. Seq2Seq cũng là mô hình Deep Learning được giới thiệu bởi nhóm nghiên cứu của Google vào năm 2014. Seq2Seq được tạo ra nhằm mục đích tạo ra một output sequence từ một input sequence mà độ dài 2 chuỗi input và output có thể khác nhau.



Mục đích ban đầu của mô hình là áp dụng vào các bài Machine Translation, tuy nhiên hiện nay Seq2Seq cũng được áp dụng vào rất nhiều các bài toán khác nhau như Nhận dạng giọng nói, Tóm tắt văn bản, Chú thích ảnh, ...

Seq2Seq gồm 2 thành phần chính: Encoder và Decoder. Cả hai thành phần này đều được hình thành từ các mạng Neural Network. Nhiệm vụ của mỗi thành phần là:
* Encoder: Chuyển đổi dữ liệu đầu vào thành một vecto đại diện có số chiều nhỏ hơn.
* Decoder: Tạo ra chuỗi đầu ra từ chuỗi đại diện được tạo ra từ Encoder

<img src="seq2seq.PNG">
<p style="text-align: center;">Hình 1: Mô hình Seq2Seq trong Machine Translation</p>

## Encoder:

Thành phần encoder chứa các mạng con là phần tử của RNN (hoặc LSTM, GRU,..) nơi nhận vào tín hiệu của một phần tử của chuỗi đầu vào và truyền tiếp về cuối mạng. <br/>
Tín hiệu đầu vào ký hiệu là x<sub>i</sub> với i là chỉ vị trí của tín hiệu đầu vào. <br/>
Các trạng thái ẩn h<sub>i</sub> được tính với công thức: h<sub>t</sub> = f(W<sup> (hh) </sup> h <sub>t-1 </sub> + W<sup> (hx) </sup>x<sub>t</sub>)

## Decoder:

Tương tự như Encoder, Decoder cũng là ngăn xếp chứa các mạng con là phần từ của RNN (hoặc LSTM, GRU, ...) sẽ dự đoán đầu ra y<sub>t</sub> tại thời điểm t. Mỗi phần tử này nhận đầu vào là trạng thái ẩn, đầu ra trước đó tạo ra kết quả đầu ra và trạng thái ẩn của chính nó.

Chuỗi đầu ra là tập các từ của câu trả lời, giả sử ta có tập các từ y<sub>i</sub> trong đó i là thứ tự của các từ.

Trạng thái ẩn h<sub>i</sub> được tính bằng công thức:
h<sub>i</sub> = f(W<sup> (hh) </sup> h<sub>t-1</sub>)

Đầu ra tại thời điểm t được tính bằng công thức: y<sub>t</sub> = softmax(W<sup>s</sup>h<sub>t</sub>)

Tùy từng bài toán cụ thể mà Encoder và Decoder sẽ sử dụng các kỹ thuật Deep Learning khác nhau. Ví dụ như trong bài toán Machine Translation thì Encoder thường là LSTM, GRU, Bidirectional RNN, còn trong bài toán Chú thích ảnh thì Encoder lại là CNN.

# 2. Attention Mechanism

Như ở phần 1 đã trình bày, Seq2Seq sử dụng Encoder để chyển đổi chuỗi đầu vào thành một vecto đại diện có độ dài cố định cho chuỗi đầu vào. Sau đó sử dụng Decoder để chuyển vecto đại diện đó thành output sequence. Với bài toán Machine Translation thì cả Encoder và Decoder đều được tạo thành từ RNN (hoặc LSTM, GRU). Chúng ta đã biết LSTM hay GRU có thể lưu trữ thông tin của chuỗi có độ dài lớn.

Chúng ta cần phải nén thông tin cần thiết vào một vecto có độ dài cố định nói trên và điều này gây khó khăn cho các mô hình mạng neural khi gặp phải những câu dài, đặc biệt là các câu có độ dài lớn hơn độ dài các câu trong tập huấn luyện. Điều này khiến cho độ chính xác của hệ thống suy giảm.

Do đó chúng ta phải sử dụng các phương pháp để tăng độ chính xác của hệ thống này. Có một số phương pháp giúp tăng độ chính xác như:
* Sử dụng Multi-layer với Bi-directional RNN
* Đảo ngược thứ tự của input sequence.
* Sử dụng input nhiều lần (feeding twice) nhưng vẫn giữ nguyên output
* Sử dụng phương pháp Attention Mechanism

Trong đó phương pháp Attention Mechanism được sử dụng nhiều nhất và giúp tăng độ chính xác đáng kể. Sau đây tôi sẽ trình bày về nguyên tắc hoạt động của phương pháp này, cụ thể là với bài toán Neural Machine Translate.

<img src="encoder-decoder-attention.PNG">
<p style="text-align: center;">Hình 2: Mô hình Seq2Seq trong Machine Translation sử dụng Attention Mechanism</p>

Theo hình 2 ta thấy sự khác biệt khi sử dụng Attention Mechanism đó là encoder không mã hóa toàn bộ câu đầu vào thành một vecto có độ dài cố định. Thay vì chỉ sử dụng context layer được tạo ra từ layer cuối cùng của Encoder, nó mã hóa câu đầu vào thành một chuỗi vecto và chọn một tập hợp con của các vecto này một cách thích hợp khi thực hiện decoder. Phương pháp này giúp mô hình xử lý tốt hơn khi gặp các câu dài.

# 3. Thuật toán

Ta có chuỗi các từ trong văn bản gốc x với độ dài n và một chuỗi văn bản được dịch tương ứng với văn bản gốc x với độ dài m:

                x = [x1, x2, ..., xn]

                y = [y1, y2, ..., ym]

p(yi∣{y1,...,yi−1},x)=g(yi−1,si,ci)
